<a href="https://colab.research.google.com/github/dtabuena/Resources/blob/main/Genetics/KEGG/Download_Kegg_Genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keggtools
import keggtools as kt
import collections
from tqdm import tqdm
import itertools

In [2]:
def get_all_pathways(ORGANISM_ID: str) -> (dict, list):

    resolver = kt.Resolver()
    org_all = resolver.get_pathway_list(ORGANISM_ID)
    path_list = list()
    for path_key in tqdm( org_all.keys() ):
        try:
            path_list.append(resolver.get_pathway(organism=ORGANISM_ID, code=path_key.split(ORGANISM_ID)[-1]))
        except: None
    return path_list

path_list = get_all_pathways("mmu")


100%|██████████| 352/352 [07:56<00:00,  1.35s/it]


In [3]:

pathway_gene_dict=dict()
for one_pathway in tqdm(path_list):
    entries_in_one_pathway = one_pathway.entries
    genes_in_all_entries_in_one_pathway = [e.name.split(" ") for e in entries_in_one_pathway if 'gene' in e.type]
    genes_in_all_entries_in_one_pathway = list(itertools.chain.from_iterable(genes_in_all_entries_in_one_pathway))
    path_name = one_pathway.name
    pathway_gene_dict[path_name] = {"title":one_pathway.title,'genes':genes_in_all_entries_in_one_pathway}
pathway_gene_dict.keys()

100%|██████████| 346/346 [00:00<00:00, 11265.65it/s]


dict_keys(['path:mmu01200', 'path:mmu01210', 'path:mmu01212', 'path:mmu01230', 'path:mmu01232', 'path:mmu01250', 'path:mmu01240', 'path:mmu00010', 'path:mmu00020', 'path:mmu00030', 'path:mmu00040', 'path:mmu00051', 'path:mmu00052', 'path:mmu00053', 'path:mmu00500', 'path:mmu00520', 'path:mmu00620', 'path:mmu00630', 'path:mmu00640', 'path:mmu00650', 'path:mmu00562', 'path:mmu00910', 'path:mmu00920', 'path:mmu00061', 'path:mmu00062', 'path:mmu00071', 'path:mmu00100', 'path:mmu00120', 'path:mmu00140', 'path:mmu00561', 'path:mmu00564', 'path:mmu00565', 'path:mmu00600', 'path:mmu00590', 'path:mmu00591', 'path:mmu00592', 'path:mmu01040', 'path:mmu00230', 'path:mmu00240', 'path:mmu00250', 'path:mmu00260', 'path:mmu00270', 'path:mmu00290', 'path:mmu00310', 'path:mmu00220', 'path:mmu00330', 'path:mmu00340', 'path:mmu00350', 'path:mmu00360', 'path:mmu00380', 'path:mmu00400', 'path:mmu00410', 'path:mmu00430', 'path:mmu00440', 'path:mmu00450', 'path:mmu00470', 'path:mmu00480', 'path:mmu00510', 'pa

In [4]:
all_gene_ids = [v['genes'] for v in pathway_gene_dict.values()]
all_gene_ids_set = set(list(itertools.chain.from_iterable(all_gene_ids)))

In [5]:
def chunk_my_list(my_list: list,chunk_size: int):
    from numpy import ceil
    n_chunks = int(ceil(len(my_list)/chunk_size))
    for c in range(0,n_chunks):
        yield [v for i,v in enumerate(my_list) if i>=c*chunk_size and i<(c+1)*chunk_size]

In [ ]:

kegg_pathway_gene_dict = dict()
for chunk in tqdm( chunk_my_list(all_gene_ids_set,50) ):
    chunk_res = kt.resolver.get_gene_names(chunk)
    for k,v in chunk_res.items():
        kegg_pathway_gene_dict[k]=v.split(";")[0]


45it [05:46,  7.80s/it]

In [ ]:
import json
from google import colab
with open(f"./pathway_gene_dict.json",'w') as outfile:
    outfile.write( json.dumps(kegg_pathway_gene_dict, indent=4) )
    colab.files.download(f"./kegg_pathway_gene_dict.json")